# Messing around with SVMs and IRL concepts

In [126]:
# Add parent directory to current path
import os.path
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.insert(0,p)


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from research.ml.svm import SVM

In [127]:
x0 = np.array([1/3, 0, 1/3, 0, 0, 0, 1/3])  # a=1,1 (Expert policy)
x1 = np.array([1/3, 1/3, 0, 0, 1/3, 0, 0])  # a=0,1
x2 = np.array([1/3, 1/3, 0, 1/3, 0, 0, 0])  # a=0,0 (Expert policly if Disparate Impact)
x3 = np.array([1/3, 0, 1/3, 0, 0, 1/3, 0])  # a=1,0
X = np.zeros((4, 7))
X[0] = x0
X[1] = x1
X[2] = x2
X[3] = x3
y = np.array([1, 0, 1, 0])  # Only 1 for expert policy

svm = SVM()
svm.fit(X, y)
display('svm.sup_idx_', svm.sup_idx_)
display('svm.sup_X_', svm.sup_X_)
display('svm.sup_y_', svm.sup_y_)
display('svm.sup_alphas', svm.sup_alphas_)

'svm.sup_idx_'

(array([0, 1, 2, 3]),)

'svm.sup_X_'

array([[0.33333333, 0.        , 0.33333333, 0.        , 0.        ,
        0.        , 0.33333333],
       [0.33333333, 0.33333333, 0.        , 0.        , 0.33333333,
        0.        , 0.        ],
       [0.33333333, 0.33333333, 0.        , 0.33333333, 0.        ,
        0.        , 0.        ],
       [0.33333333, 0.        , 0.33333333, 0.        , 0.        ,
        0.33333333, 0.        ]])

'svm.sup_y_'

array([ 1, -1,  1, -1])

'svm.sup_alphas'

array([8.99999849, 8.99999803, 9.00000031, 9.00000077])

In [3]:
display(np.round(svm.predict_proba(X), 3))

array([ 1., -0.,  1., -0.])

In [4]:
svm._compute_discriminant(np.array([x3]))

array([-1.00001581])

In [5]:
"""
In [Abbeel, Ng, 2004] they say "the vector w^(i) we want is the unit vector
orthogonal to the maximum margin separating hyperplane." I believe that unit
vector is the (dot product? projection?) of svm.sup_X_ and svm.sup_alphas.The
reason I think that is because the discriminant for an input point is computed
by multiplying sup_X_ by sup_alphas_ by the input point, and then the value of
the discriminant determines whether its within the H+/- hyperplane boundary or
not. I'm interpreting "sup_X_" as "the points/directions that matter" (ie the
actual support vectors) and "sup_alphas_" as "how much each of those
points/directions matter".

Translating this into our IRL problem, the "sup_X_" represent "the policies
that matter" and the "sup_alphas_" represent how much each policy matters.

So "points" are "policies", and "directions/magnitudes" are "feature
expectations". So the unit vector orthogonal to the maximum margin separating
hyperplane (ie w^(i)) will be a k-length vector (k is size of phi) that, when
computed as w^(i)^T dot (some_policy), represents the distance of that policy
to the expert policy. That w^(i) also represents the learned reward function.

Therefore, our learned reward function should be a k-length vector, and should
be obtained from the svm by:
```
# <1, n_sup_vectors> <n_sup_vectors, k>
w = np.dot(svm.sup_alphas_.T, svm.sup_X_)
```
"""
display(np.dot(svm.sup_alphas_.T, svm.sup_X_))


"""
If I'm interpreting this right, the w represents the reward function that best
separates the expert policy from the non-expert policies. I.e. it yields the
largest difference between the expert reward and non-expert rewards. What I'm
confused by is that this vector is used to compute the discriminant. So when
this vector is multiplied by an input policy, if it's large, then we know the
input policy is not the experts. So this is like an inverse reward...

Oh wait. When this value is large AND POSITIVE, that means that it's likely
the expert's. So need to add in the y part to the discriminant computation.

```
w = np.dot(svm.sup_alphas.T, (svm.sup_X_ * svm.sup_y_))
```
"""
sup_X_ = np.array(svm.sup_X_)

for i in range(len(svm.sup_y_)):
    sup_X_[i] *= svm.sup_y_[i]

display(np.round(np.dot(svm.sup_alphas_.T, sup_X_), 3))

array([12.00009674,  6.00004932,  6.00004742,  3.00002302,  3.00002629,
        3.00002208,  3.00002535])

array([ 0., -0.,  0.,  3., -3., -3.,  3.])

In [6]:
"""
Trial 1
    When we exclude the 1,0 policy as a training point, we see the following
    learned reward values:
    
        [-0.   , -1.715,  1.715, -0.858, -0.857,  0.   ,  1.715]
        
    which shows that the 3rd state and last states are equally desirable because
    they are able to separate the expert from non-expert policies equally.

Trial 2
    When we include the 1,0 policy as a negative training point (non-expert),
    thereby only getting positive reward when both individuals receive a positive
    prediction, we see the following learned rewards:
    
        [-0. , -1. ,  1. , -0.5, -0.5, -2.5,  3.5]
        
    Which shows that the 3rd state is no longer considered as desirable, because
    now there is a policy that has this state but is not the expert's.

Trial 3
    When we include the 0,0 policy as a positive training point, thereby
    implementing disparate impact as our reward function, we see the following
    learned rewards:
    
        [-0.,  0., -0.,  3., -3., -3.,  3.]
        
    So it values the final state as the most significant, which makes sense since
    we cannot determine the desirability of the intermediate states until the
    final state is observed.
"""
pass

# Full IRL – 03/07/2023

In [10]:
# Add parent directory to current path. Needed for research imports.
import os.path
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.insert(0,p)


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from research.ml.svm import SVM
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (
    train_test_split, KFold, RandomizedSearchCV)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
# Import adult dataset
adult = pd.read_csv('./../../data/adult.csv')
adult.sample(3)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
23371,41,Self-emp-inc,194636,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,99999,0,65,United-States,>50K
26679,43,Private,397963,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,594,0,16,United-States,<=50K
12591,51,Private,345459,Some-college,10,Never-married,Exec-managerial,Unmarried,Black,Female,0,0,40,United-States,<=50K


### Notes and Assumptions

* Meaning of `fnlwgt` column: it is the (estimated) number of people each row in the data represents. I'm removing it for now. Although we may want to consider resampling each row based on its `fnlwgt` value. See what other papers do with this column.
* There are 5 possible values for `race`. I'm going to make it binary with `White` vs `Non-White`. Gender only has male and female so leaving this as is.

In [56]:
df = adult.copy()

# Transform sensitive attriburtes and target to binary values
df['race'] = df['race'] == 'White'
df['gender'] = df['gender'] == 'Male'
df['income'] = df['income'] == '>50K'
display('Race', df['race'].value_counts(1))
display('Gender', df['gender'].value_counts(1))
display('Income', df['income'].value_counts(1))

'Race'

True     0.855043
False    0.144957
Name: race, dtype: float64

'Gender'

True     0.668482
False    0.331518
Name: gender, dtype: float64

'Income'

False    0.760718
True     0.239282
Name: income, dtype: float64

In [57]:
feature_types = {
    'boolean': [
        'race', 'gender',
    ],
    'categoric': [
        'workclass', 'education', 'marital-status', 'occupation',
        'relationship', 'native-country',
    ],
    'continuous': [
        'age', 'educational-num', 'capital-gain', 'capital-loss',
        'hours-per-week',
    ],
    'meta': [
        'fnlwgt'
    ],
}

In [58]:
# Drop meta columns
df = df.drop(columns=feature_types['meta'])
X = df[feature_types['boolean'] + feature_types['categoric'] + feature_types['continuous']]
y = df['income']

In [59]:
##
# Helper functions
##
def train_clf(feature_types, clf_inst, X_train, y_train):
    """
    Create demonstration (X, yhat, y)
    
    Parameters
    ----------
    feature_types : dict
    clf_inst : sklearn classifier
        Unfitted sklearn classifier instance. E.g. `RandomForestClassifier()`.
    X_train : pandas.DataFrame
    y_train : pandas.Series
    
    Returns
    -------
    clf : sklearn classifier
        Fitted classifier.
    """
    numeric_trf = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]
    )
    categoric_trf = Pipeline(
        steps=[
            ("encoder", OneHotEncoder(handle_unknown="ignore")),
            ("selector", SelectPercentile(chi2, percentile=50)),
        ]
    )
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_trf, feature_types['continuous']),
            ("cat", categoric_trf, feature_types['categoric']),
        ]
    )
    clf = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", clf_inst),
        ]
    )
    clf.fit(X_train, y_train)
    return clf
    
    
def generate_demo(clf, X_test, y_test):
    """
    Create demonstration (X, yhat, y) from a fitted classifer `clf`.
    
    Parameters
    ----------
    clf : fitted sklearn classifier
    X_test : pandas.DataFrame
    y_test : pandas.Series
    
    Returns
    -------
    demo : pandas.DataFrame
        Demonstrations. Each demonstration represents an iteration of a
        trained classifier and its predictions on a hold-out set. Columns are
            **`X` columns : all input columns (i.e. `X`)
            yhat : predictions
            y : ground truth targets
    """
    yhat = clf.predict(X_test)
    demo = pd.DataFrame(X_test)
    demo['yhat'] = yhat
    demo['y'] = y_test
    return demo


def feature_exp(demo, z_col):
    """
    Transform demonstrations into feature expectations
    
    Parameters
    ----------
    demo : pandas.DataFrame
        Demonstrations. Each demonstration represents an iteration of a
        trained classifier and its predictions on a hold-out set. Columns are
            **`X` columns : all input columns (i.e. `X`)
            yhat : predictions
            y : ground truth targets
    z_col : str
        The column to use for fairness computations. E.g. "race".
            
    Returns
    -------
    array<float>, len(2)
        mu0 : float, range(0,1)
            The accuracy feature expectations.
        mu1 : float, range(0,1)
            The fairness (disparate impact) feature expectations.
    """
    # Accuracy
    mu0 = np.mean(demo['yhat'] == demo['y'])
    # Disparate Impact
    p_yhat_eq_1_giv_z_eq_0 = ((demo['yhat'] == 1) & (demo[z_col] == 0)).mean()
    p_yhat_eq_1_giv_z_eq_1 = ((demo['yhat'] == 1) & (demo[z_col] == 1)).mean()
    mu1 = 1 - max([
        p_yhat_eq_1_giv_z_eq_0 - p_yhat_eq_1_giv_z_eq_1,
        p_yhat_eq_1_giv_z_eq_1 - p_yhat_eq_1_giv_z_eq_0,
    ])
    return np.array([mu0, mu1])

In [60]:
"""
Split data into two: (X_demo, y_demo), (X_irl_valid, y_irl_valid)
- The former is used for generating demonstrations.
- The latter is used for computing the error term in the IRL loop.
"""
X_demo, X_irl_valid, y_demo, y_irl_valid = train_test_split(
    X, y, test_size=.33)

del X, y  # Make sure I don't acidentally use these variables later on

In [68]:
m = 3  # Number of demonstrations to generate
muE = np.zeros((m, 2))  # expert demo feature expectations

# Generate demonstrations (populate muE)
k_fold = KFold(m)
for k, (train, test) in enumerate(k_fold.split(X_demo, y_demo)):
    print(f"Staring iteration {k+1}/{m}")
    X_train, y_train = X_demo.iloc[train], y_demo.iloc[train]
    X_test, y_test = X_demo.iloc[test], y_demo.iloc[test]
    print('\tFitting classifier...')
    clf = train_clf(feature_types, RandomForestClassifier(), X_train, y_train)
    print('\tGenerating demo...')
    demo = generate_demo(clf, X_test, y_test)
    print('\tComputing feature expectations...')
    muE[k] = feature_exp(demo, z_col='race')
    print(f"\tmuE[{k}]: {muE[k]}")
    
display(muE)

Staring iteration 1/3
	Fitting classifier...
	Generating demo...
	Computing feature expectations...
	muE[0]: [0.83965897 0.82269894]
Staring iteration 2/3
	Fitting classifier...
	Generating demo...
	Computing feature expectations...
	muE[1]: [0.84946828 0.82984965]
Staring iteration 3/3
	Fitting classifier...
	Generating demo...
	Computing feature expectations...
	muE[2]: [0.85038504 0.81949028]


array([[0.83965897, 0.82269894],
       [0.84946828, 0.82984965],
       [0.85038504, 0.81949028]])

In [69]:
# Generate initial policy(s)
n_init_policies = 1
mu = []

for i in range(n_init_policies):
    print(f"Staring iteration {i+1}/{n_init_policies}")
    X_train, X_test, y_train, y_test = train_test_split(
        X_demo, y_demo, test_size=.33)
    print('\tFitting classifier...')
    dummy_clf = DummyClassifier(strategy="uniform")
    clf = train_clf(feature_types, dummy_clf, X_train, y_train)
    print('\tGenerating demo...')
    demo = generate_demo(clf, X_test, y_test)
    print('\tComputing feature expectations...')
    mu.append(feature_exp(demo, z_col='race'))
    
display(mu)

Staring iteration 1/1
	Fitting classifier...
	Generating demo...
	Computing feature expectations...


[array([0.49541624, 0.65367164])]

In [70]:
def irl_error(w, muE, muj):
    """
    Computes t[i] = wT(muE-mu[j])
    """
    mu_delta = muE.mean(axis=0) - muj.mean(axis=0)
    err = np.dot(w, mu_delta)
    return err, mu_delta

In [96]:
from sklearn.metrics         import make_scorer
from sklearn.model_selection import GridSearchCV


def custom_reward_function(y, y_pred):
    # Only care about accuracy
    acc = np.mean(y == y_pred)
    return acc


def learn_policy(weights, X_train, y_train):
    """
    Compute a policy (classifier) from a set of weights. The weights come from
    the SVM classifier used to distinguish the expert demonstrations from the
    learned policies.
    
    weights : array<float>
        Weights from SVM classifier.
    X_train : pandas.DataFrame
    y_train : pandas.Series
        
    Returns
    -------
    clf_pol : sklearn compatible classifier
    """
    custom_scorer = make_scorer(
        custom_reward_function, greater_is_better=True)
    param_grid = {
        'max_depth': [20, 100],
        'max_features': [2, 3],
        'min_samples_leaf': [1, 5],
        'min_samples_split': [2, 8],
        'n_estimators': [10, 100]
    }
    rf = RandomForestClassifier()
    clf_pol = GridSearchCV(
        estimator=rf, param_grid=param_grid, cv=3, scoring=custom_scorer)
    clf_pol = train_clf(feature_types, clf_pol, X_train, y_train)
    return clf_pol

In [105]:
"""
Run IRL loop.
Create a clf dataset where inputs are feature expectations and outputs are
whether the policy is expert or learned through IRL iterations. Then train an
SVM classifier on this dataset. Then extract the weights of the svm and use
them as the weights for the "reward" function. Then use this reward function
to learn a policy (classifier). Then compute the feature expectations from
this classifer on the irl hold-out set. Then compute the error between the
feature expectations of this learned clf and the demonstration feature exp. If
this error is less than epsilon, stop. The reward function is the final set of
weights.
"""
X_irl_exp = pd.DataFrame(muE, columns=['acc', 'disp_imp'])
y_irl_exp = pd.Series(np.ones(m), dtype=bool)
X_irl_learn = pd.DataFrame(mu, columns=['acc', 'disp_imp'])
y_irl_learn = pd.Series(np.zeros(len(mu)), dtype=bool)
X_irl = pd.concat([X_irl_exp, X_irl_learn], axis=0)
y_irl = pd.concat([y_irl_exp, y_irl_learn], axis=0).astype(int)

epsilon = .05
t = []  # Errors for each iteration
weights = []
i = 0
max_iter = 5
while True:
    print(f"Starting iteration {i+1}/{max_iter} ...")
    
    # Train SVM classifier
    svm = SVM().fit(X_irl, y_irl)
    
    # Extract the weights from the SVM classifier
    wi = svm.weights()
    weights.append(wi)
    
    # TODO: Learn a policy (clf) from the reward (svm weights)
    pol_clf = learn_policy(weights, X_demo.iloc[0:10_000], y_demo.iloc[0:10_000])
    
    # Compute feature expectations of the learned policy
    demo = generate_demo(pol_clf, X_irl_valid, y_irl_valid)
    muj = feature_exp(demo, z_col='race')
    
    # Append policy's feature expectations to irl clf dataset
    X_irl_learn_i = pd.DataFrame(np.array([muj]), columns=['acc', 'disp_imp'])
    y_irl_learn_i = pd.Series(np.zeros(1), dtype=int)
    X_irl = pd.concat([X_irl, X_irl_learn_i], axis=0)
    y_irl = pd.concat([y_irl, y_irl_learn_i], axis=0)
    
    # Compute error of the learned policy: t[i] = wT(muE-mu[j])
    ti, mu_delta = irl_error(wi, muE, muj)
    t.append(ti)
    print(f"\tt[i] = {t[i]:.2f} \t weights[{i}] = {np.round(weights[i], 3)}")
    
#     if ti < epsilon or i >= max_iter - 1:
    if i >= max_iter - 1:
        break
        
    i += 1
    
# The weights should converge towards [1, 0])

Starting iteration 1/5 ...
	t[i] = -0.12 	 weights[0] = [4.681 2.299]
Starting iteration 2/5 ...
	t[i] = 3.24 	 weights[1] = [  54.541 -108.531]
Starting iteration 3/5 ...
	t[i] = 2.88 	 weights[2] = [  59.344 -107.733]
Starting iteration 4/5 ...
	t[i] = 2.87 	 weights[3] = [  57.697 -104.954]
Starting iteration 5/5 ...
	t[i] = 3.32 	 weights[4] = [  64.085 -116.052]
